# Ship Detection

This notebook demonstrates how to use the geoai package for ship detection using a pre-trained model.

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/ship_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [1]:
%pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.3 MB/s eta 0:00:00
   

## Import libraries

In [1]:
import geoai

## Download sample data

In [2]:
raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/ships_dubai.tif"
)
raster_path = geoai.download_file(raster_url, "ships_dubai.tif")

File already exists: ships_dubai.tif


## Visualize data

In [3]:
geoai.view_raster(raster_url)

## Initialize model

In [4]:
detector = geoai.ShipDetector()

Model path not specified, downloading from Hugging Face...


ship_detection.pth:   0%|          | 0.00/176M [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--giswqs--geoai/snapshots/19e7bc566118360fc01e3a56837008452aa3a9df/ship_detection.pth
Model loaded successfully


## Generate masks

In [7]:
output_path = "ships_dubai_masks.tif"

In [6]:
masks_path = detector.generate_masks(
    raster_path,
    output_path=output_path,
    confidence_threshold=0.9,
    mask_threshold=0.7,
    overlap=0.25,
    chip_size=(256, 256),
    batch_size=4,
)

Dataset initialized with 21 rows and 39 columns of chips
Image dimensions: 7457 x 3912 pixels
Chip size: 256 x 256 pixels
Overlap: 25.0% (stride_x=192, stride_y=192)
CRS: EPSG:3857
Processing raster with 205 batches


100%|██████████| 205/205 [01:29<00:00,  2.29it/s]


Masks with confidence values saved to ships_dubai_masks.tif


## Vectorize masks

In [8]:
gdf = detector.vectorize_masks(
    output_path,
    output_path="ships_dubai_masks.geojson",
    confidence_threshold=0.8,
    min_object_area=100,
    max_object_size=10000,
)

Processing masks from: ships_dubai_masks.tif
Found 538 connected components


Processing components: 100%|██████████| 538/538 [00:19<00:00, 27.20it/s]


Saved 442 objects with confidence to ships_dubai_masks.geojson


## Visualize initial results

In [9]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Calculate geometric properties

In [10]:
gdf = geoai.add_geometric_properties(gdf)
gdf.head()

,geometry,confidence,class,area_m2,length_m,perimeter_m,area_bbox_m2,area_convex_m2,area_filled_m2,major_length_m,minor_length_m,eccentricity,orientation,elongation,extent,solidity,complexity
0,"POLYGON ((6137683.482 2890333.01, 6137722.594 ...",0.921569,1,1109.321560,134.949299,134.949299,1109.321560,1109.321560,1109.321560,39.111850,28.362799,0.688568,0.000000,1.378984,1.000000,1.0,1.142976
1,"POLYGON ((6138297.254 2890315.473, 6138329.316...",0.986820,1,260.982641,81.836470,81.836470,527.583981,260.982641,260.982641,33.012732,7.905519,0.970904,13.781192,4.175910,0.494675,1.0,1.429013
2,"POLYGON ((6138332.716 2890313.341, 6138366.8 2...",0.996078,1,672.192295,108.038363,108.038363,934.263516,672.192295,672.192295,34.581183,19.438217,0.827067,9.727346,1.779031,0.719489,1.0,1.175510
3,"POLYGON ((6138250.532 2890308.425, 6138284.63 ...",0.985730,1,352.105228,89.233320,89.233320,513.438050,352.105228,352.105228,34.373094,10.243656,0.954562,7.263021,3.355549,0.685779,1.0,1.341485
4,"POLYGON ((6137310.431 2890300.742, 6137313.881...",0.949020,1,24.099754,19.991122,19.991122,48.558027,24.099754,24.099754,5.934928,4.060734,0.729286,54.461804,1.461541,0.496308,1.0,1.148751


In [11]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Filter results

In [12]:
m = geoai.view_raster(raster_url, backend="ipyleaflet")
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

Use the drawing tool to select the area of interest.

In [ ]:
aoi = m.user_rois

if aoi is None:

    aoi = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {},
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [55.133729, 25.110277],
                            [55.134072, 25.11393],
                            [55.134823, 25.115601],
                            [55.136025, 25.117116],
                            [55.137677, 25.118127],
                            [55.140145, 25.118787],
                            [55.142248, 25.11902],
                            [55.142012, 25.118243],
                            [55.140831, 25.116728],
                            [55.13948, 25.116903],
                            [55.137956, 25.116825],
                            [55.136132, 25.115543],
                            [55.13566, 25.114416],
                            [55.135467, 25.1136],
                            [55.135939, 25.112609],
                            [55.136218, 25.111657],
                            [55.13551, 25.110685],
                            [55.134373, 25.110102],
                            [55.133729, 25.110277],
                        ]
                    ],
                },
            }
        ],
    }

In [ ]:
import geopandas as gpd

In [ ]:
aoi_gdf = gpd.GeoDataFrame.from_features(aoi["features"], crs="EPSG:4326").to_crs(
    gdf.crs
)

Intersect the selected area with the vectorized masks to filter the results.

In [ ]:
gdf_filter = gdf[gdf.intersects(aoi_gdf.geometry[0])]
gdf_filter.head()

## Visualize final results

In [ ]:
geoai.view_vector_interactive(gdf_filter, column="confidence", tiles=raster_url)

## Save results

In [ ]:
gdf_filter.to_file("ships_dubai.geojson")

![image](https://github.com/user-attachments/assets/80df3827-8453-45b2-af89-21662fdf95a6)